This is a simlpe cuGraph example of calculating "Page Rank" for NCAAW basketball teams based on the wins and losses in a single season. We treat two teams as having a "link" between them if they've played a game against each other, with the "link" proceeding from the losing team to the winning team.

I've used cuGraph to calculate Page Rank not becasue I need a lot of speed - we are dealing here with a very small dataset. But primarily becasue I am familiar with it, and it's part of Rapids library, whcih I am all too eager to promote. ;) Unfortunately, cuGraph is still not available in Kaggle Docker, so I have to install it from skrach here from Chris Deotte's dataset.

In [ ]:
import sys
!cp ../input/rapids/rapids.0.18.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import cugraph

In [ ]:
import cupy as cp
import cuml, cudf
from sklearn.model_selection import train_test_split 

from cuml.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [ ]:
WRSCResults = cudf.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WRegularSeasonCompactResults.csv')
WTeams = cudf.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WTeams.csv')
WRSCResults.head()

In [ ]:
WTeams.head()

In [ ]:
WRSCResults_2021 = WRSCResults[WRSCResults['Season'] == 2021]
WRSCResults_2021.head()

In [ ]:
G1 = cugraph.Graph()
G1.from_cudf_edgelist(WRSCResults_2021, source='LTeamID', destination='WTeamID')

In [ ]:
pr1 = cugraph.pagerank(G1, alpha = 0.85, max_iter = 500, tol = 1.0e-05)
pr1.columns = ['pagerank', 'TeamID']
pr1 = pr1.merge(WTeams)
pr1 = pr1.sort_values(by='pagerank',  ascending=False).reset_index(drop=True)
pr1.head(10)

We see that some of the top teams are all relatively unknown, and not really otherwise that highly ranked. In other words, this measure doesn't seem to have a lot of "face validity". This was not a very strong signal, but when I looked at my feature importances it usually ranked in the top 10.